# Notebook 01 · Batch Lakehouse Pipeline: Bronze → Silver → Gold

## Purpose

This notebook builds the core batch data pipeline for FunnelPulse. It takes raw e-commerce event logs for October and November, and turns them into a layered lakehouse:

- **Bronze**: Raw event store, normalized and partitioned for scalable access  
- **Silver**: Clean, deduplicated, analytics ready events  
- **Gold**: Aggregated funnel metrics for different business views  

All later components of the system, including streaming and anomaly detection, depend on the tables created here.

---

## Input Data

**Source dataset**

- Kaggle: *eCommerce Events History in Cosmetics Shop*  
- Raw files provided as monthly CSVs (for example `2019-Oct.csv`, `2019-Nov.csv`)  
- Each row represents a product level user interaction with fields such as:
  - `event_time`, `event_type` (view, cart, purchase, etc.)
  - `user_id`, `user_session`
  - `product_id`, `category_id`, `category_code`, `brand`
  - `price`

**Scope in this notebook**

- Ingests data for **October and November**  
- Uses incremental month by month ingestion so the design mirrors production where new data arrives continuously

---

## High Level Workflow

1. **Environment and project paths**
   - Initialize a Spark session following the course setup
   - Define the project root and key directories:
     - `data_raw/` for temporary CSV uploads
     - `tables/` for Parquet based lakehouse tables

2. **Bronze creation (incremental)**
   - Ingest one month of raw CSV events at a time  
   - Convert time fields and derive partitioning keys  
   - Append each month into a unified bronze table

3. **Silver creation**
   - Read all bronze events for October and November  
   - Apply cleaning, normalization and deduplication  
   - Write a partitioned silver table suitable for analytical queries

4. **Gold creation**
   - Read the silver table  
   - Build the first core gold table:
     - Hourly funnel metrics grouped by brand

5. **Sanity checks and sample analytics**
   - Inspect schema and sample rows
   - Run simple summary queries to verify that metrics and relationships look reasonable

---

## Bronze Layer · Raw Event Store

**What it represents**

The bronze layer is the first structured storage of raw events. It is designed to look like the landing zone of a production data platform:

- Rows are close to the original Kaggle schema
- Only basic normalization is applied so that downstream jobs have a consistent starting point

**Key actions in this notebook**

- Read a single month CSV from `data_raw/`
- Cast `event_time` from string into a timestamp type
- Derive `event_date` as the calendar date of `event_time`
- Write all events into `tables/bronze_events` as Parquet
  - Use `event_date` as a partition column
  - Use **append** mode so multiple months can be ingested incrementally

**Why this matters**

- Partitioning by date makes later scans efficient as the dataset grows
- Incremental ingestion models a production feed where new days or months arrive continuously
- Bronze is the single source of truth for historical event logs in the platform

---

## Silver Layer · Cleaned and Analytics Ready Events

**What it represents**

The silver layer standardizes and cleans data from bronze so that analytical and machine learning workloads do not have to handle raw quirks directly.

**Key transformations**

- **Filtering**
  - Remove rows with missing or non positive prices
  - Remove rows with missing `event_time` or `event_type`
- **Normalization**
  - Create `brand_norm` as a lowercased version of `brand`
  - Create `category_code_norm` as a sanitized, lowercased version of `category_code` that is safe to use as a dimension
- **Data quality flags**
  - Mark events with missing session, brand or category fields
- **Deduplication**
  - Drop exact duplicates using a composite key of:
    - `event_time`, `user_id`, `user_session`, `product_id`, `event_type`
- **Storage**
  - Write clean events to `tables/silver_events` as Parquet
  - Partition by `event_date` for scalable time based access

**Why this matters**

- Silver provides a stable, trusted event table that downstream jobs can rely on
- It separates raw ingestion concerns from analytical modeling concerns
- The deduped, normalized schema is the basis for all funnel and anomaly logic

---

## Gold Layer · Hourly Funnel Metrics by Brand

**What it represents**

Gold tables hold pre aggregated metrics tailored to specific business questions. In this notebook we build the first and most important one:

- **`gold_funnel_hourly_brand`**  
  - Grain: one brand per one hour window  
  - Metrics:
    - `views`           number of view events  
    - `carts`           number of cart events  
    - `purchases`       number of purchase events  
    - `revenue`         total purchase value in that hour  
    - `view_to_cart_rate`      carts per view  
    - `cart_to_purchase_rate`  purchases per cart  
    - `conversion_rate`        purchases per view  
  - Partitioned by `window_date` for efficient access by day

**Workflow**

- Read `silver_events`
- Group events by:
  - 1 hour window on `event_time`
  - `brand_norm`
- Aggregate counts and revenue
- Derive funnel rates with protection against divide by zero
- Store the results as a gold table in `tables/gold_funnel_hourly_brand`

**Why this matters**

- This table is the backbone of FunnelPulse
- It is the input for both:
  - Descriptive funnel analytics in dashboards
  - Anomaly detection and alerting logic in later notebooks
- It gives an executive ready view of how each brand is performing in near real time at hourly resolution

---

## Role of This Notebook in the Overall System

This notebook is the **foundation of the entire project**:

- It turns raw CSV logs into a structured lakehouse with bronze, silver and gold layers
- It defines the core funnel metrics that the rest of the system depends on
- It prepares data for:
  - Additional gold tables (daily by brand, category, price band)  
  - Streaming pipelines that compute the same metrics over a simulated real time feed  
  - Anomaly detection on top of hourly conversion behavior  

All subsequent notebooks build on top of the tables created here rather than going back to the raw CSV files.

In [1]:
# CELL 1: Spark initialization for local development
# Works on macOS/Linux without JupyterHub

import os
import sys

# Set JAVA_HOME to Java 17 (required for PySpark 3.4+)
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"

# Add parent directory to path to import config
sys.path.insert(0, os.path.dirname(os.getcwd()))

from pyspark.sql import SparkSession

# Create Spark session for local execution
spark = (
    SparkSession.builder
    .appName("FunnelPulse Batch Pipeline")
    .master("local[*]")
    .config("spark.driver.memory", "4g")
    .config("spark.sql.shuffle.partitions", "200")
    .getOrCreate()
)

print(spark)
print(f"Spark UI available at: http://localhost:4040")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/27 07:25:16 WARN Utils: Your hostname, Aranyas-Laptop.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.180 instead (on interface en0)
25/11/27 07:25:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/27 07:25:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI available at: http://localhost:4040


In [2]:
# CELL 2: Project paths configuration
# Uses the project directory (parent of notebooks folder)

import os

# Get project root (parent directory of notebooks)
project_root = os.path.dirname(os.getcwd())

raw_dir    = os.path.join(project_root, "data_raw")
tables_dir = os.path.join(project_root, "tables")

bronze_path = os.path.join(tables_dir, "bronze_events")
silver_path = os.path.join(tables_dir, "silver_events")
gold_path   = os.path.join(tables_dir, "gold_funnel_hourly_brand")

# Ensure directories exist
os.makedirs(raw_dir, exist_ok=True)
os.makedirs(tables_dir, exist_ok=True)

print("Project root:", project_root)
print("Raw dir     :", raw_dir)
print("Bronze path :", bronze_path)
print("Silver path :", silver_path)
print("Gold path   :", gold_path)

Project root: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project
Raw dir     : /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/data_raw
Bronze path : /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/bronze_events
Silver path : /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/silver_events
Gold path   : /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/gold_funnel_hourly_brand


In [3]:
# CELL 3: Ingest 2019-Oct CSV month into BRONZE (incremental)

from pyspark.sql.functions import to_timestamp, to_date, col

# 1) Name of the file you've uploaded to data_raw
single_csv_name = "2019-Oct.csv"   # change this for each month later
single_csv_path = os.path.join(raw_dir, single_csv_name)

print("Reading single CSV:", single_csv_path)

df_raw = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(single_csv_path)
)

print("Raw schema:")
df_raw.printSchema()
df_raw.show(5, truncate=False)

# 2) Normalize event_time and add event_date for partitioning
df_bronze_month = (
    df_raw
    .withColumn("event_time", to_timestamp("event_time"))
    .withColumn("event_date", to_date(col("event_time")))
)

# 3) Decide write mode: overwrite if bronze doesn't exist yet, append otherwise
import pathlib

bronze_exists = pathlib.Path(bronze_path).exists()
write_mode = "append" if bronze_exists else "overwrite"

print("Bronze exists:", bronze_exists)
print("Writing this month to BRONZE with mode:", write_mode)

(
    df_bronze_month
    .write
    .mode(write_mode)
    .partitionBy("event_date")
    .parquet(bronze_path)
)

print("Finished writing to BRONZE at:", bronze_path)

# 4) Quick check on total bronze rows so far
bronze_check = spark.read.parquet(bronze_path)
print("Total BRONZE rows (all ingested months so far):", bronze_check.count())
bronze_check.show(5, truncate=False)

Reading single CSV: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/data_raw/2019-Oct.csv


Raw schema:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+-------------+------+-----+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code|brand |price|user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-------------+------+-----+---------+------------------------------------+
|2019-09-30 20:00:00|cart      |5773203   |1487580005134238553|NULL         |runail|2.62 |463240011|26dd6e6e-4dac-4778-8d2c-92e149dab885|
|2019-09-30 20:00:03|cart      |5773353   |1487580005134

Finished writing to BRONZE at: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/bronze_events
Total BRONZE rows (all ingested months so far): 4102283
+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+----------+
|event_time         |event_type|product_id|category_id        |category_code|brand   |price|user_id  |user_session                        |event_date|
+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+----------+
|2019-10-02 00:00:00|view      |5877451   |1487580006300255120|NULL         |jessnail|44.29|544257140|2828b81d-89b3-4b25-96ae-15292222f338|2019-10-02|
|2019-10-02 00:00:00|cart      |5692467   |1487580007852147670|NULL         |staleks |3.57 |555853251|46af5723-07da-4e6c-ad08-932f141a1410|2019-10-02|
|2019-10-02 00:00:02|view      |5822415   |1487580013229244601|NULL 

In [4]:
# CELL 3.1: Ingest 2019-Nov CSV month into BRONZE (incremental)

from pyspark.sql.functions import to_timestamp, to_date, col

# 1) Name of the file you've uploaded to data_raw
single_csv_name = "2019-Nov.csv"   # change this for each month later
single_csv_path = os.path.join(raw_dir, single_csv_name)

print("Reading single CSV:", single_csv_path)

df_raw = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(single_csv_path)
)

print("Raw schema:")
df_raw.printSchema()
df_raw.show(5, truncate=False)

# 2) Normalize event_time and add event_date for partitioning
df_bronze_month = (
    df_raw
    .withColumn("event_time", to_timestamp("event_time"))
    .withColumn("event_date", to_date(col("event_time")))
)

# 3) Decide write mode: overwrite if bronze doesn't exist yet, append otherwise
import pathlib

bronze_exists = pathlib.Path(bronze_path).exists()
write_mode = "append" if bronze_exists else "overwrite"

print("Bronze exists:", bronze_exists)
print("Writing this month to BRONZE with mode:", write_mode)

(
    df_bronze_month
    .write
    .mode(write_mode)
    .partitionBy("event_date")
    .parquet(bronze_path)
)

print("Finished writing to BRONZE at:", bronze_path)

# 4) Quick check on total bronze rows so far
bronze_check = spark.read.parquet(bronze_path)
print("Total BRONZE rows (all ingested months so far):", bronze_check.count())
bronze_check.show(5, truncate=False)

Reading single CSV: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/data_raw/2019-Nov.csv


Raw schema:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+
|event_time         |event_type      |product_id|category_id        |category_code|brand   |price|user_id  |user_session                        |
+-------------------+----------------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+
|2019-10-31 20:00:02|view            |5802432   |1487580009286598681|NULL         |NULL    |0.32 |562076640|09fafd6c-6c99-46b1-834f-33527f4de241|
|2019-10-31 20:00:09|car

Finished writing to BRONZE at: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/bronze_events
Total BRONZE rows (all ingested months so far): 8738120
+-------------------+----------+----------+-------------------+-------------+-------+-----+---------+------------------------------------+----------+
|event_time         |event_type|product_id|category_id        |category_code|brand  |price|user_id  |user_session                        |event_date|
+-------------------+----------+----------+-------------------+-------------+-------+-----+---------+------------------------------------+----------+
|2019-11-22 00:00:00|cart      |5794074   |1487580005511725929|NULL         |NULL   |5.27 |574742033|8c95bf39-00cb-40da-b050-9886ae81571a|2019-11-22|
|2019-11-22 00:00:00|view      |5857360   |1487580008145748965|NULL         |runail |1.43 |567776789|c3e54af9-7253-435e-be00-3957dde96758|2019-11-22|
|2019-11-22 00:00:00|cart      |5843548   |1487580010628776014|NULL      

In [5]:
# CELL 4: build SILVER (cleaned, normalized, DQ flags, deduped)

from pyspark.sql.functions import lower, regexp_replace, when

print("Reading bronze table from:", bronze_path)
bronze = spark.read.parquet(bronze_path)

print("Bronze row count:", bronze.count())
bronze.show(5, truncate=False)

silver = bronze

# Drop obviously bad rows: null/<=0 price, null event_time or event_type
silver = silver.filter(silver.price.isNotNull() & (silver.price > 0))
silver = silver.filter(silver.event_time.isNotNull() & silver.event_type.isNotNull())

# Normalize brand and category_code
silver = (
    silver
    .withColumn("brand_norm", lower(col("brand")))
    .withColumn("category_code_norm", lower(col("category_code")))
    .withColumn(
        "category_code_norm",
        regexp_replace(col("category_code_norm"), "[^a-z0-9\\.]", "_")
    )
)

# Simple data quality flags
silver = (
    silver
    .withColumn("dq_missing_session", silver.user_session.isNull())
    .withColumn("dq_missing_brand", silver.brand.isNull())
    .withColumn("dq_missing_category", silver.category_code.isNull())
)

print("Silver schema after cleaning:")
silver.printSchema()

silver.select(
    "event_time", "event_type",
    "brand", "brand_norm",
    "category_code", "category_code_norm",
    "dq_missing_session", "dq_missing_brand"
).show(5, truncate=False)

# Deduplicate on composite key
dedup_cols = ["event_time", "user_id", "user_session", "product_id", "event_type"]
silver_dedup = silver.dropDuplicates(dedup_cols)

print("Row count before dedup:", silver.count())
print("Row count after  dedup:", silver_dedup.count())

# Write silver, partitioned by event_date (same as bronze)
(
    silver_dedup
    .write
    .mode("overwrite")
    .partitionBy("event_date")
    .parquet(silver_path)
)

print("Silver table written to:", silver_path)

# Quick check
silver_check = spark.read.parquet(silver_path)
print("Silver check row count:", silver_check.count())
silver_check.show(5, truncate=False)

Reading bronze table from: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/bronze_events
Bronze row count: 8738120
+-------------------+----------+----------+-------------------+-------------+-------+-----+---------+------------------------------------+----------+
|event_time         |event_type|product_id|category_id        |category_code|brand  |price|user_id  |user_session                        |event_date|
+-------------------+----------+----------+-------------------+-------------+-------+-----+---------+------------------------------------+----------+
|2019-11-22 00:00:00|cart      |5794074   |1487580005511725929|NULL         |NULL   |5.27 |574742033|8c95bf39-00cb-40da-b050-9886ae81571a|2019-11-22|
|2019-11-22 00:00:00|view      |5857360   |1487580008145748965|NULL         |runail |1.43 |567776789|c3e54af9-7253-435e-be00-3957dde96758|2019-11-22|
|2019-11-22 00:00:00|cart      |5843548   |1487580010628776014|NULL         |NULL   |3.57 |531053364|9153fb

+-------------------+----------+-------+----------+-------------+------------------+------------------+----------------+
|event_time         |event_type|brand  |brand_norm|category_code|category_code_norm|dq_missing_session|dq_missing_brand|
+-------------------+----------+-------+----------+-------------+------------------+------------------+----------------+
|2019-11-22 00:00:00|cart      |NULL   |NULL      |NULL         |NULL              |false             |true            |
|2019-11-22 00:00:00|view      |runail |runail    |NULL         |NULL              |false             |false           |
|2019-11-22 00:00:00|cart      |NULL   |NULL      |NULL         |NULL              |false             |true            |
|2019-11-22 00:00:01|view      |grattol|grattol   |NULL         |NULL              |false             |false           |
|2019-11-22 00:00:01|cart      |irisk  |irisk     |NULL         |NULL              |false             |false           |
+-------------------+----------+

Row count after  dedup: 8260755


Silver table written to: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/silver_events
Silver check row count: 8260755
+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+----------+------------------+------------------+----------------+-------------------+----------+
|event_time         |event_type|product_id|category_id        |category_code|brand   |price|user_id  |user_session                        |brand_norm|category_code_norm|dq_missing_session|dq_missing_brand|dq_missing_category|event_date|
+-------------------+----------+----------+-------------------+-------------+--------+-----+---------+------------------------------------+----------+------------------+------------------+----------------+-------------------+----------+
|2019-11-22 11:28:38|view      |5734492   |1487580005050352469|NULL         |haruyama|3.76 |228337277|000caab0-80fa-43ef-8abc-327ae98f2358|har

In [6]:
# CELL 5: build GOLD table (hourly funnel metrics by brand)

from pyspark.sql.functions import window, sum as _sum, to_date

silver = spark.read.parquet(silver_path)

print("Silver row count:", silver.count())

# Group by 1-hour windows and brand
windowed = (
    silver
    .groupBy(
        window(col("event_time"), "1 hour").alias("w"),
        col("brand_norm").alias("brand")
    )
    .agg(
        _sum((col("event_type") == "view").cast("int")).alias("views"),
        _sum((col("event_type") == "cart").cast("int")).alias("carts"),
        _sum((col("event_type") == "purchase").cast("int")).alias("purchases"),
        _sum(
            when(col("event_type") == "purchase", col("price")).otherwise(0.0)
        ).alias("revenue")
    )
)

# Flatten the window struct
gold = (
    windowed
    .withColumn("window_start", col("w.start"))
    .withColumn("window_end", col("w.end"))
    .drop("w")
)

# Funnel rates with divide-by-zero protection
gold = (
    gold
    .withColumn(
        "view_to_cart_rate",
        when(col("views") > 0, col("carts") / col("views"))
    )
    .withColumn(
        "cart_to_purchase_rate",
        when(col("carts") > 0, col("purchases") / col("carts"))
    )
    .withColumn(
        "conversion_rate",
        when(col("views") > 0, col("purchases") / col("views"))
    )
)

# Partition by date of window_start
gold = gold.withColumn("window_date", to_date(col("window_start")))

gold.printSchema()
gold.orderBy("window_start", "brand").show(20, truncate=False)

# Write gold table
(
    gold
    .write
    .mode("overwrite")
    .partitionBy("window_date")
    .parquet(gold_path)
)

print("Gold table written to:", gold_path)

Silver row count: 8260755
root
 |-- brand: string (nullable = true)
 |-- views: long (nullable = true)
 |-- carts: long (nullable = true)
 |-- purchases: long (nullable = true)
 |-- revenue: double (nullable = true)
 |-- window_start: timestamp (nullable = true)
 |-- window_end: timestamp (nullable = true)
 |-- view_to_cart_rate: double (nullable = true)
 |-- cart_to_purchase_rate: double (nullable = true)
 |-- conversion_rate: double (nullable = true)
 |-- window_date: date (nullable = true)



+-----------+-----+-----+---------+-----------------+-------------------+-------------------+-------------------+---------------------+-------------------+-----------+
|brand      |views|carts|purchases|revenue          |window_start       |window_end         |view_to_cart_rate  |cart_to_purchase_rate|conversion_rate    |window_date|
+-----------+-----+-----+---------+-----------------+-------------------+-------------------+-------------------+---------------------+-------------------+-----------+
|NULL       |202  |185  |59       |284.0199999999999|2019-09-30 20:00:00|2019-09-30 21:00:00|0.9158415841584159 |0.31891891891891894  |0.29207920792079206|2019-09-30 |
|airnails   |0    |2    |2        |2.38             |2019-09-30 20:00:00|2019-09-30 21:00:00|NULL               |1.0                  |NULL               |2019-09-30 |
|ardell     |1    |0    |0        |0.0              |2019-09-30 20:00:00|2019-09-30 21:00:00|0.0                |NULL                 |0.0                |2019-

Gold table written to: /Users/aranyaaryaman/Desktop/bigData 2/finalProject/Big-Data-Project/tables/gold_funnel_hourly_brand


In [7]:
# CELL 6: verify GOLD and do a simple analytic query

gold_check = spark.read.parquet(gold_path)

print("Gold rows:", gold_check.count())

# Show a few rows ordered by time and brand
gold_check.orderBy("window_start", "brand").show(50, truncate=False)

# Example: top 10 brands by total revenue across all windows
from pyspark.sql.functions import desc

gold_check.groupBy("brand").agg(
    _sum("revenue").alias("total_revenue")
).orderBy(desc("total_revenue")).show(10, truncate=False)

Gold rows: 183145
+-----------+-----+-----+---------+-----------------+-------------------+-------------------+-------------------+---------------------+-------------------+-----------+
|brand      |views|carts|purchases|revenue          |window_start       |window_end         |view_to_cart_rate  |cart_to_purchase_rate|conversion_rate    |window_date|
+-----------+-----+-----+---------+-----------------+-------------------+-------------------+-------------------+---------------------+-------------------+-----------+
|NULL       |202  |185  |59       |284.0199999999999|2019-09-30 20:00:00|2019-09-30 21:00:00|0.9158415841584159 |0.31891891891891894  |0.29207920792079206|2019-09-30 |
|airnails   |0    |2    |2        |2.38             |2019-09-30 20:00:00|2019-09-30 21:00:00|NULL               |1.0                  |NULL               |2019-09-30 |
|ardell     |1    |0    |0        |0.0              |2019-09-30 20:00:00|2019-09-30 21:00:00|0.0                |NULL                 |0.0    